In [1]:
pip install pymongo ollama 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\harsh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Check if MongoDB is running and print all documents in a collection
from pymongo import MongoClient
Client = MongoClient("mongodb://localhost:27017/")
db = Client["product_db"]  # change as needed
collection = db["products"]  # change as needed

for doc in collection.find():
    print(doc)

{'_id': ObjectId('6837002e8264cee68ed0505d'), 'ProductID': 101, 'ProductName': 'Wireless Mouse', 'Category': 'Electronics', 'Price': 25.99, 'Rating': 4.5, 'ReviewCount': 200, 'Stock': 150, 'Discount': '10%', 'Brand': 'Logitech', 'LaunchDate': datetime.datetime(2022, 1, 15, 0, 0)}
{'_id': ObjectId('6837002e8264cee68ed0505e'), 'ProductID': 102, 'ProductName': 'Gaming Keyboard', 'Category': 'Electronics', 'Price': 75.49, 'Rating': 4.7, 'ReviewCount': 350, 'Stock': 85, 'Discount': '5%', 'Brand': 'Corsair', 'LaunchDate': datetime.datetime(2021, 11, 20, 0, 0)}
{'_id': ObjectId('6837002e8264cee68ed0505f'), 'ProductID': 103, 'ProductName': 'Noise Cancelling Headphones', 'Category': 'Electronics', 'Price': 199.99, 'Rating': 4.8, 'ReviewCount': 125, 'Stock': 60, 'Discount': '15%', 'Brand': 'Sony', 'LaunchDate': datetime.datetime(2021, 9, 5, 0, 0)}
{'_id': ObjectId('6837002e8264cee68ed05060'), 'ProductID': 104, 'ProductName': 'Running Shoes', 'Category': 'Sports', 'Price': 49.99, 'Rating': 4.3, '

In [3]:
# Test case: 1
"""Find all products with a rating below 4.5 that have more than 200 reviews and
are offered by the brand 'Nike' or 'Sony'"""
# Test case: 2
"""Which products in the Electronics category have a rating of 4.5 or higher and are
in stock?"""
# Test case: 3
"""List products launched after January 1, 2022, in the Home & Kitchen or Sports
categories with a discount of 10% or more, sorted by price in descending order."""

'List products launched after January 1, 2022, in the Home & Kitchen or Sports\ncategories with a discount of 10% or more, sorted by price in descending order.'

source code


In [ ]:
import json
import os
from pymongo import MongoClient
from ollama import Client

# Connect to local MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["product_db"]  # change as needed
collection = db["products"]      # change as needed

# Connect to local Ollama LLM (Mistral)
llm_client = Client(host="http://localhost:11434")

def generate_mongo_query(user_input):
    system_msg = {
        "role": "system",
        "content": (
            "You are a MongoDB query generator. Convert natural language into MongoDB query **filters** only.\n"
            "Return ONLY the Python dict to be used inside collection.find(FILTER), NOT sort, projection, or options.\n"
            "Do NOT return multiple objects. Do NOT include sorting, projections, or aggregations.\n"
            "Only use these fields with correct types:\n"
            "ProductID (string), ProductName (string), Category (string), Price (number), Rating (number),\n"
            "ReviewCount (number), Stock (number), Discount (number), Brand (string), LaunchDate (ISODate string).\n"
            "Use valid operators: $gt, $lt, $gte, $lte, $in, $regex, etc.\n"
            "Example input: Products from Home & Kitchen or Sports launched after Jan 1, 2022, with 10%+ discount\n"
            "Example output: {\"Category\": {\"$in\": [\"Home & Kitchen\", \"Sports\"]}, \"LaunchDate\": {\"$gte\": \"2022-01-01T00:00:00\"}, \"Discount\": {\"$gte\": 0.1}}"
        )
    }

    user_msg = {
        "role": "user",
        "content": user_input
    }

    response = llm_client.chat(model="mistral", messages=[system_msg, user_msg])

    try:
        content = response['message']['content'].strip()
        if content.startswith("```"):
            content = content.strip("```python\n").strip("```")
        query_dict = json.loads(content.replace("'", "\""))
        return query_dict
    except Exception as e:
        print("Error parsing query from model:", e)
        print("Model response was:", response['message']['content'])
        return {}

def main():
    user_input = input("Enter your query (natural language): ")
    
    query = generate_mongo_query(user_input)
    print("Generated MongoDB Query:", query)

    if not query:
        print("Invalid query. Exiting.")
        return

    # Fetch data
    results = list(collection.find(query))
    if not results:
        print("No results found.")
        return

    # Ask user what to do with the results
    action = input("Do you want to (d)isplay the results or (s)ave to a file? (d/s): ").strip().lower()

    if action == 'd':
        for doc in results:
            print(json.dumps(doc, indent=4, default=str))
    elif action == 's':
        output_path = input("Enter output file name (e.g., output.json): ").strip()
        output_data = {
            "query": user_input,
            "mongodb_query": query,
            "results": results
        }
        with open(output_path, "w") as f:
            json.dump(output_data, f, indent=4, default=str)
        print(f"Query, MongoDB filter, and results saved to {output_path}")
    else:
        print("Invalid option.")

if __name__ == "__main__":
    main()
